In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import requests 

In [73]:
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
# import folium 

# TASK 1

In [3]:
link = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
BFS = BeautifulSoup(link, 'xml')
BFS_tab = BFS.find('table')
columns = ['Postalcode' ,'Borough' ,'Neighborhood']
df = pd.DataFrame(columns=columns)

In [5]:
for x in BFS_tab.find_all('tr'):
    list_yip=[]
    for k in x.find_all('td'):
        list_yip.append(k.text.strip())
    if len(list_yip)==3:
        df.loc[len(df)] = list_yip

In [6]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df_new = df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_new = df_new.reset_index(drop=False)
df_new.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [8]:
df = df.merge(df_new, how='left', on='Postalcode')

In [9]:
df.drop(['Neighborhood'],axis=1,inplace=True)

In [10]:
df = df.rename(columns={'Neighborhood_joined':'Neighborhood'})

In [11]:
df = df.rename(columns={'Neighborhood_joined':'Neighborhood'})

In [12]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
df.shape

(180, 3)

In [14]:
df = df[df.Borough != 'Not assigned']

In [15]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [17]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
df.shape

(103, 3)

# TASK 2

In [19]:
#reading the file to coord dataframe
df['Latitude'] = '0';
df['Longitude'] = '0';

cordinates = pd.read_csv('https://cocl.us/Geospatial_data')

In [20]:
for indx in df.index:
    for indxcord in cordinates.index:
        if df.iloc[indx, 0] == cordinates.iloc[indxcord, 0]:
            df.iloc[indx, 3] = cordinates.iloc[indxcord, 1]
            df.iloc[indx, 4] = cordinates.iloc[indxcord, 2]          

In [21]:
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


# TASK 3

In [22]:
df.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [23]:
tab_analise = df.loc[df.Borough.isin(['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto'])].copy()

In [24]:
tab_analise.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
15,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
19,M4E,East Toronto,The Beaches,43.6764,-79.293


In [25]:
tab_analise.shape

(39, 5)

In [26]:
import folium

In [27]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(tab_analise['Latitude'],tab_analise['Longitude'],tab_analise['Borough'],tab_analise['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
CLIENT_ID = 'PXXFRZXNDT3HAAGE2PTLXXZTSNVD0PKRE41KMGDP3JOSRDN5'
CLIENT_SECRET = 'UO2TGIOWALSIT1QK4XLNIZYT2NJFNN4KC5HDGSJVRRCFYAY1' 
VERSION = '20190101' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PXXFRZXNDT3HAAGE2PTLXXZTSNVD0PKRE41KMGDP3JOSRDN5
CLIENT_SECRET:UO2TGIOWALSIT1QK4XLNIZYT2NJFNN4KC5HDGSJVRRCFYAY1


In [30]:
toronto_data = df.copy()

In [31]:
toronto_venues = getNearbyVenues(toronto_data.Neighborhood, toronto_data.Latitude, toronto_data.Longitude)
toronto_venues.head(5)

,Neighborhood,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [32]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(5)

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0

In [34]:
toronto_grouped.shape

(95, 238)

In [36]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4         Mac & Cheese Joint  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3     Coffee Shop  0.14
4             Pub  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0                Bank  0.09
1         Coffee Shop  0.09
2  Frozen Yogurt Shop  0.04
3          Restaurant  0.04
4    Sushi Restaurant  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.07
1      Sandwich Place  0.07
2          Restaurant  0.07
3           J

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =toronto_grouped['Neighborhood']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Farm,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Sandwich Place,Coffee Shop,Pub,Distribution Center,Dim Sum Restaurant,Diner,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Mobile Phone Shop,Sandwich Place,Diner,Restaurant,Deli / Bodega,Ice Cream Shop,Supermarket
3,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Juice Bar,Italian Restaurant,Restaurant,Sandwich Place,Pizza Place,Liquor Store,Boutique,Comfort Food Restaurant,Locksmith


In [67]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [68]:
len(kmeans.labels_)

95

In [69]:
toronto_merged = toronto_data
toronto_merged['Cluster Labels'] = pd.DataFrame(kmeans.labels_)
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head(100)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7533,-79.3297,0.0,Bus Stop,Park,Food & Drink Shop,Women's Store,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.7259,-79.3156,0.0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,0.0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Yoga Studio,Restaurant,Pub,Chocolate Shop,Mexican Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648,0.0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Coffee Shop,Vietnamese Restaurant,Event Space,Accessories Store,Miscellaneous Shop,Ethiopian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,0.0,Coffee Shop,Diner,Gym,Sandwich Place,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944,0.0,Fast Food Restaurant,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
7,M3B,North York,Don Mills,43.7459,-79.3522,0.0,Gym,Japanese Restaurant,Beer Store,Restaurant,Café,Coffee Shop,Asian Restaurant,Chinese Restaurant,Sandwich Place,Bike Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099,0.0,Pizza Place,Pet Store,Athletics & Sports,Pharmacy,Café,Breakfast Spot,Intersection,Bank,Fast Food Restaurant,Gastropub
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,0.0,Café,Tea Room,Clothing Store,Theater,Sandwich Place,Fast Food Restaurant,Ramen Restaurant,Steakhouse,Plaza,Diner


In [43]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='capstone_project')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [48]:
cluster

0.0

In [71]:
# toronto_merged['Cluster Labels'].fillna(5, inplace = True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

In [72]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [61]:
toronto_merged[toronto_merged['Cluster Labels']==0].head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7533,-79.3297,0,Bus Stop,Park,Food & Drink Shop,Women's Store,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.7259,-79.3156,0,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Yoga Studio,Restaurant,Pub,Chocolate Shop,Mexican Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648,0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Coffee Shop,Vietnamese Restaurant,Event Space,Accessories Store,Miscellaneous Shop,Ethiopian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,0,Coffee Shop,Diner,Gym,Sandwich Place,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944,0,Fast Food Restaurant,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
7,M3B,North York,Don Mills,43.7459,-79.3522,0,Gym,Japanese Restaurant,Beer Store,Restaurant,Café,Coffee Shop,Asian Restaurant,Chinese Restaurant,Sandwich Place,Bike Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099,0,Pizza Place,Pet Store,Athletics & Sports,Pharmacy,Café,Breakfast Spot,Intersection,Bank,Fast Food Restaurant,Gastropub
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,0,Café,Tea Room,Clothing Store,Theater,Sandwich Place,Fast Food Restaurant,Ramen Restaurant,Steakhouse,Plaza,Diner


In [62]:
toronto_merged[toronto_merged['Cluster Labels']==1]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.6864,-79.4,1,Pub,Coffee Shop,Pizza Place,American Restaurant,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Restaurant,Supermarket,Bank


In [63]:
toronto_merged[toronto_merged['Cluster Labels']==2]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.7138,-79.4901,2,Park,Construction & Landscaping,Bakery,Basketball Court,Trail,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant,Drugstore


In [64]:
toronto_merged[toronto_merged['Cluster Labels']==3]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M6B,North York,Glencairn,43.7096,-79.4451,3,Japanese Restaurant,Pizza Place,Asian Restaurant,Pub,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
26,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,3,Gas Station,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Drugstore,Donut Shop
46,M3L,North York,Downsview,43.739,-79.5069,3,Grocery Store,Park,Business Service,Shopping Mall,Athletics & Sports,Airport,Gym / Fitness Center,Bank,Discount Store,Electronics Store
50,M9L,North York,Humber Summit,43.7563,-79.566,3,Gym,Home Service,Dim Sum Restaurant,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
58,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648,3,College Stadium,Café,Skating Rink,General Entertainment,Farm,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
65,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.7574,-79.2733,3,Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Chinese Restaurant,Pet Store,Electronics Store,Eastern European Restaurant,Drugstore,Dessert Shop,Dog Run
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.6056,-79.5013,3,Gym,Hobby Shop,Fast Food Restaurant,Café,Flower Shop,Restaurant,Bakery,Mexican Restaurant,Fried Chicken Joint,Liquor Store
94,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.7067,-79.5941,3,Drugstore,Bar,Rental Car Location,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [65]:
toronto_merged[toronto_merged['Cluster Labels']==4]

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,M2P,North York,York Mills West,43.7528,-79.4,4,Park,Convenience Store,Electronics Store,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Eastern European Restaurant,Drugstore


In [66]:
grpby=toronto_merged[['Neighborhood','Cluster Labels']].groupby('Cluster Labels')
grpby.count().sort_values('Neighborhood', ascending=False)

,Neighborhood
Cluster Labels,
0,92
3,8
1,1
2,1
4,1
